In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

##### 各ベンチマークの実行時間を記録する処理
# # Benchmarks = []
# # for benchmark in benchmarks:
# #     Benchmarks.append(benchmark.upper())
# # print(Benchmarks)

# Processes = [8, 16, 32, 64, 128, 256]
# for process in Processes:
#     TimeDataList = return_TimeDataList(process)
#     for DictData in TimeDataList:
#         benchmark = DictData.benchmark
#         process = DictData.process
#         BenchmarkClass = DictData.BenchmarkClass
#         time = DictData.time
#         FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# # print(TimeDataList[0].return_AllData())


In [ ]:
def GenerateMapeTable(FixedClass="B", FixedProcess="64", test_ratio=0.3):
    
    DirName = "./table_LatexForm/"
    FileSuffix = f"Train0{int(test_ratio*10)}.table"
    FilePrefix = "MapeTableFixed"
    
    input_MapeTableDF = return_MapeTableDataframe_FixedProcess(FixedProcess=FixedProcess, test_ratio=test_ratio)
    return_MapeTableDF = return_MapeTableOnlyStrDataframe(input_MapeTableDF=input_MapeTableDF)
    return_MapeTableDF.to_latex(f"{DirName}{FilePrefix}{FixedProcess}{FileSuffix}")
    input_MapeTableDF = return_MapeTableDataframe_FixedClass(FixedClass=FixedClass,test_ratio=test_ratio)
    return_MapeTableDF = return_MapeTableOnlyStrDataframe(input_MapeTableDF=input_MapeTableDF)
    return_MapeTableDF.to_latex(f"{DirName}{FilePrefix}{FixedClass}{FileSuffix}")

# MAPE表を生成する例   
# GenerateMapeTable()

In [ ]:
### 関数：ConvertIndexNameToNumOfProfile()
### 引数：行名が使用したプロファイル, 列名が平均誤差率・予測コスト・実測コストとなっているデータフレーム
### 返値：
def ConvertIndexNameToNumOfProfile(inputDF, Fixed="Process"):
    index = inputDF.index.tolist()
    # ConvertedIndexには入力データフレームの行名から使用されたプロファイルの数を格納している。
    ConvertedIndex = [0] * len(index)
    for i in range(len(index)):
        if(Fixed=="Process"):
            ConvertedIndex[i] = len(convert_StrListToStrList(index[i]))
        elif(Fixed=="Class"):
            ConvertedIndex[i] = len(convert_StrListToIntList(index[i]))
    returnDF = inputDF.copy(deep=True)
    returnDF.index = ConvertedIndex
    return(returnDF)

In [ ]:
def GenGraphAveragePerProfileNum(benchmarks=["cg"], Fixed="Process", Fix=64, Predict="D", EnableTitle=False):
    plt.figure()
    for benchmark in benchmarks:
        if(benchmark == "is" or benchmark=="ft" or benchmark=="lu"):
            continue
        if(Fixed=="Process"):
            FixedDF = return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark, FixProcess=Fix, predict_class=Predict)
            GraphTitle = f"ベンチマーク{benchmark}で実行プロセス数を固定"
        else:
            FixedDF = return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark, FixClass=Fix, predict_process=Predict)
            GraphTitle = f"ベンチマーク{benchmark}で実行プロセス数を固定"
        ConvertedIndexFixedDF = ConvertIndexNameToNumOfProfile(FixedDF, Fixed=Fixed)
        x = ConvertedIndexFixedDF.index.tolist()
        y = ConvertedIndexFixedDF["Average"].tolist()
        plt.plot(x,y,label=f"{benchmark}")
        plt.legend()
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("平均誤差率")
    if(EnableTitle):
        plt.title(GraphTitle)
    plt.show()

##使用例
# GenGraphAveragePerProfileNum(benchmarks=benchmarks, Fixed="Process", Fix=64, Predict="D")
# GenGraphAveragePerProfileNum(benchmarks=benchmarks, Fixed="Class", Fix="B", Predict=256)

In [ ]:
# for benchmark in benchmarks:
#     print(return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark, FixClass="B", predict_process=256))
def GenGraphTotalTimePerProfileNum(benchmark="cg", Fixed="Process", Fix=64, Predict="D", EnableTitle=False):
    if(Fixed=="Process"):
        FixedDF = return_ErrorRateFixedProcess_AveragePredictCostRealCost(benchmark=benchmark, FixProcess=Fix, predict_class=Predict)
        GraphTitle = f"ベンチマーク{benchmark}で実行プロセス数を固定"
    else:
        FixedDF = return_ErrorRateFixedClass_AveragePredictCostRealCost(benchmark=benchmark, FixClass=Fix, predict_process=Predict)
        GraphTitle = f"ベンチマーク{benchmark}でベンチマーククラスを固定"
    ConvertedIndexFixedDF = ConvertIndexNameToNumOfProfile(inputDF=FixedDF, Fixed="Process")
    x = ConvertedIndexFixedDF.index.tolist()
    y = ConvertedIndexFixedDF["PredictCosts"]
    RealCost = ConvertedIndexFixedDF["RealCost"]
    plt.figure()
    plt.scatter(x, y, color="blue")
    plt.plot(x, RealCost, color="red")
    plt.xlabel("使用したプロファイル数")
    plt.ylabel("総計算時間")
    if(EnableTitle):
        plt.title(GraphTitle)
    plt.show()

### 使用例
# GenGraphTotalTimePerProfileNum(EnableTitle=True)
# GenGraphTotalTimePerProfileNum(benchmark=benchmark, Fixed="Class", Fix="C", Predict=256, EnableTitle=True)